# Interactive time-gating

## Preliminaries

### Imports

In [ ]:
%matplotlib notebook
#%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from ipywidgets import interact, interactive, IntSlider, IntText, Dropdown, fixed, FloatSlider

import numpy as np
import pandas
from scipy.ndimage import convolve1d
from scipy import signal, special

from PyDynamic.uncertainty.propagate_DFT import GUM_iDFT, GUM_DFT, DFT2AmpPhase, AmpPhase2DFT
from PyDynamic.misc import complex_2_real_imag as c2ri
from PyDynamic.misc import real_imag_2_complex as ri2c

### Data Loading

#### Load Only Main Diagonal of Uncertainty Information

In [ ]:
# load reflection data
file_reflection = "../data/Beatty Line s11 MagPhase data.xlsx"
df = pandas.read_excel(file_reflection, skiprows=2)

# add missing 0Hz-frequency point and construct complex variables
# phase for a 0Hz is assumed to be zero
f_old = np.r_[0, df.iloc[:, 0]]  # GHz
s11_mag_old = np.r_[df.iloc[0, 1], df.iloc[:, 1]]
s11_mag_unc_old = np.r_[df.iloc[0, 2], df.iloc[:, 2]]
s11_phase_old = np.r_[0, df.iloc[:, 3]] / 180 * np.pi
s11_phase_unc_old = np.r_[0, df.iloc[:, 4]] / 180 * np.pi

# translate into PyDynamic-internal Re/Im-representation
#s11_UAP =  np.square(np.r_[s11_mag_unc, s11_phase_unc])
#s11_ri, s11_ri_cov = AmpPhase2DFT(s11_mag, s11_phase, s11_UAP)


#### Get Full Statistical Covariance from Individual Experiments

In [ ]:
file_reflection = "../data/Beatty Line New Type-A Re Im Data.xlsx"
dfs = pandas.read_excel(file_reflection, skiprows=1, sheet_name=None)
df0 = dfs[list(dfs.keys())[0]]

f = np.r_[0, df0.iloc[:, 0]]  # GHz

# load raw data from all individual experiments (separate sheets)
s11_runs_raw = np.array([df.iloc[:, 1] + 1j * df.iloc[:, 2] for df in dfs.values()])

# include new datapoint at 0Hz which is required for discrete Fourier transform
s11_runs_0Hz = np.atleast_2d(np.absolute(s11_runs_raw[:,0])).T
s11_runs = np.concatenate([s11_runs_0Hz, s11_runs_raw], axis=1)

# convert data into PyDynamic-real-imag representation
s11_ri_runs = c2ri(s11_runs)

# get mean and covariance from enhanced raw data
s11_ri = np.mean(s11_ri_runs, axis=0)
s11_ri_cov = np.cov(s11_ri_runs, rowvar=False)

# get magnitude-phase representation for representation purposes
s11_mag, s11_phase, s11_UAP = DFT2AmpPhase(s11_ri, s11_ri_cov)
s11_mag_unc = np.sqrt(np.diag(s11_UAP)[:len(s11_mag)])
s11_phase_unc = np.sqrt(np.diag(s11_UAP)[len(s11_mag):])

#### Comparison of Statistical Uncertainty Component and Full Uncertainty Information

In [ ]:
# visualize raw input
fig_comp_raw, ax_comp_raw = plt.subplots(nrows=4, figsize=(8, 8), tight_layout=True)
ax_comp_raw[0].plot(f, s11_mag, label="statistical cov.", color="tab:gray", linewidth=4)
ax_comp_raw[0].plot(f_old, s11_mag_old, label="given values", color="tab:red")

ax_comp_raw[1].semilogy(f, s11_mag_unc, label="statistical cov.", color="tab:gray", linewidth=4)
ax_comp_raw[1].semilogy(f_old, s11_mag_unc_old, label="given values", color="tab:red")

ax_comp_raw[2].plot(f, np.rad2deg(s11_phase), label="statistical cov.", color="tab:gray", linewidth=4)
ax_comp_raw[2].plot(f_old, np.rad2deg(s11_phase_old), label="given values", color="tab:red")

ax_comp_raw[3].semilogy(f, np.rad2deg(s11_phase_unc), label="statistical cov.", color="tab:gray", linewidth=4)
ax_comp_raw[3].semilogy(f_old, np.rad2deg(s11_phase_unc_old), label="given values", color="tab:red")

ax_comp_raw[0].legend()
ax_comp_raw[0].set_title("Frequency Domain")
ax_comp_raw[0].set_ylabel("magnitude [-]")
ax_comp_raw[1].set_ylabel("magnitude unc [-]")
ax_comp_raw[2].set_ylabel("phase [°]")
ax_comp_raw[3].set_ylabel("phase unc [°]")
ax_comp_raw[3].set_xlabel("f [GHz]")

In [ ]:
#plt.plot(np.rad2deg(s11_phase) - np.rad2deg(s11_phase_old))
#plt.plot(s11_mag - s11_mag_old)

### Visualize Input Data

#### Raw Mag/Phase Frequency Domain Data

In [ ]:
# visualize raw input
fig_in, ax_in = plt.subplots(nrows=4, figsize=(8, 8), tight_layout=True)
ax_in[0].plot(f, s11_mag, label="s11", color="tab:gray")
ax_in[1].plot(f, s11_mag_unc, label="s11", color="tab:gray")
ax_in[2].plot(f, np.rad2deg(s11_phase), label="s11", color="tab:gray")
ax_in[3].plot(f, np.rad2deg(s11_phase_unc), label="s11", color="tab:gray")

ax_in[0].legend()
ax_in[0].set_title("Frequency Domain")
ax_in[0].set_ylabel("magnitude [-]")
ax_in[1].set_ylabel("magnitude unc [-]")
ax_in[2].set_ylabel("phase [°]")
ax_in[3].set_ylabel("phase unc [°]")
ax_in[3].set_xlabel("f [GHz]")

#### Transformation into Time Domain (Application of Inverse DFT)

In [ ]:
# convert reflection data to time domain 
Nx = 2*len(s11_mag) - 1
S11, S11_cov = GUM_iDFT(s11_ri, s11_ri_cov, Nx=Nx)

# provide timestamps
# same as: dt = 0.5/np.max(f); t = np.arange(0, stop = 1/np.median(np.diff(f)) + dt, step = dt)
t = np.linspace(0.0, 1/np.median(np.diff(f)), num=Nx)

In [ ]:
# visualize time domain data
fig_in_td, ax_in_td = plt.subplots(nrows=3, figsize=(8, 10), tight_layout=True, sharex=False, gridspec_kw={'height_ratios':(1,1,2)})
maxi = np.max(np.abs(S11_cov))
cnorm = colors.SymLogNorm(vmin=-maxi, vmax=maxi, linthresh=1e-14)

ax_in_td[0].get_shared_x_axes().join(ax_in_td[0], ax_in_td[1])
ax_in_td[0].plot(t, S11, label="signal", color="tab:gray")
ax_in_td[1].semilogy(t, np.sqrt(np.diag(S11_cov)), label="signal unc", color="tab:gray")
img0 = ax_in_td[2].imshow(S11_cov,  cmap="PuOr", norm=cnorm)

fig_in_td.colorbar(img0, ax=ax_in_td[2])
ax_in_td[0].legend()
ax_in_td[0].set_title("Time Domain")
ax_in_td[0].set_ylabel("signal [-]")
ax_in_td[1].set_ylabel("signal unc [-]")
ax_in_td[1].set_xlabel("t [ns]")
ax_in_td[2].set_title("Covariance of time signal")

ax_in_td[0].set_ylim(-0.25, 0.25)
#ax_in_td[1].set_ylim(8e-5, 10e-5)

# add inset value-plot
ax_in_td_0_inset = ax_in_td[0].inset_axes([0.25, 0.65, 0.30, 0.30])
ax_in_td_0_inset.plot(t, S11, color="tab:gray")
ax_in_td_0_inset.set_xlim(0.0, 0.8)
ax_in_td[0].indicate_inset_zoom(ax_in_td_0_inset, edgecolor="red")

# add inset unc-plot
ax_in_td_1_inset = ax_in_td[1].inset_axes([0.55, 0.65, 0.30, 0.30])
ax_in_td_1_inset.plot(t, np.sqrt(np.diag(S11_cov)), color="tab:gray")
ax_in_td_1_inset.set_xlim(0.0, 0.8)
ax_in_td_1_inset.set_ylim(1.0e-5, 6.0e-4)
ax_in_td_1_inset.set_yscale("log")
ax_in_td[1].indicate_inset_zoom(ax_in_td_1_inset, edgecolor="red")

In [ ]:
# visualize time domain magnitude data
fig_in_td2, ax_in_td2 = plt.subplots(nrows=1, figsize=(8, 6), tight_layout=True)

ax_in_td2.plot(t, np.abs(S11), label="signal magnitude", color="tab:gray")

ax_in_td2.set_xlim(0.0, 0.8)
ax_in_td2.legend()
ax_in_td2.set_title("Time Domain")
ax_in_td2.set_ylabel("signal magnitude [-]")
ax_in_td2.set_xlabel("t [ns]")

#### Check that Chaining iDFT + DFT Results in the Original Data

This shows that the low uncertainties in the time domain are just a consequence of 
the (inverse) Fourier transformation. 

In [ ]:
s11_ri_idtest, s11_ri_cov_idtest = GUM_DFT(S11, S11_cov)

print("(Numerical) identity is achieved for the real-imag-representation:")
print("real/imag representation identity: ", np.allclose(s11_ri_idtest, s11_ri))
print("Covariance identity:               ", np.allclose(s11_ri_cov_idtest, s11_ri_cov))


s11_mag_idtest, s11_phase_idtest, s11_UAP_idtest = DFT2AmpPhase(s11_ri_idtest, s11_ri_cov_idtest, tol=1e-15)
s11_mag_unc_idtest = np.sqrt(np.diag(s11_UAP_idtest)[:len(s11_mag_idtest)])
s11_phase_unc_idtest = np.sqrt(np.diag(s11_UAP_idtest)[len(s11_mag_idtest):])

print("\n\n(Numerical) identity is achieved for the mag-phase-representation: (except some elements with a mag/phase ratio close to zero)")
print("magnitude identity:     ", np.allclose(s11_mag_idtest, s11_mag))
print("magnitude unc identity: ", np.allclose(s11_mag_unc_idtest, s11_mag_unc))
print("phase identity:         ", np.allclose(s11_phase_idtest, s11_phase))
print("phase unc identity:     ", np.allclose(s11_phase_unc_idtest, s11_phase_unc))

## Define the Time-Gate

### Useful Functions and Initializations

In [ ]:
def window(kind = "kaiser", order = 6, width = 100):
    return signal.get_window((kind, order), width, fftbins=False)

def gate(window, window_start, total_length):
    if window_start +  len(window) > total_length or window_start < 0:
        raise ValueError("Cannot place window.")

    pad_pre = int(window_start)
    pad_post = int(total_length - len(window) - pad_pre)
    gate = np.r_[np.zeros(pad_pre), window, np.zeros(pad_post)]
    
    # heuristic model for gate unc, probably too complicated :-)
    gate_unc = 0.0 * 1e-5*np.abs(signal.filtfilt(*signal.butter(1, 0.30, "lowpass"), np.abs(np.diff(np.r_[gate, 0])), padlen=100)) 

    return gate, gate_unc

# setup of window/gate (would need some heuristic)
window_kind = "kaiser"
window_order = 0
window_width = 400

initial_window = window(window_kind, window_order, window_width)
initial_gate, initial_gate_unc = gate(initial_window, 100, len(t))


### Position the Gate

#### Interactive Positioning

In [ ]:
fig_gate, ax_gate = plt.subplots(nrows=2, figsize=(8, 8), tight_layout=True, sharex=True)

# setup secondary y-axis
ax2_gate = [None, None]
ax2_gate[0] = ax_gate[0].twinx()
ax2_gate[1] = ax_gate[1].twinx()

# plot signal and gate
line_signal, = ax_gate[0].plot(t, S11, label="signal", color="tab:gray")
line_gate, = ax2_gate[0].plot(t, initial_gate, label="gate", color="r")

# plot uncertainty of signal and gate
ax_gate[1].plot(t, np.sqrt(np.diag(S11_cov)), label="signal unc", color="tab:gray")
line_gate_unc, = ax2_gate[1].plot(t, initial_gate_unc, label="gate unc", color="r")

# decorate
ax_gate[0].legend()
#ax_gate[1].set_ylim(8e-5, 10e-5)

ax_gate[0].set_ylabel("signal [-]")
ax2_gate[0].set_ylabel("gate [-]")
ax_gate[1].set_xlabel("t [ns]")

ax_gate[0].yaxis.get_label().set_color(line_signal.get_color())
ax2_gate[0].yaxis.get_label().set_color(line_gate.get_color())

def update(window_start, window_width, window_order):
    
    w = window(window_kind, window_order, window_width)
    g, ug = gate(w, window_start, len(t))

    line_gate.set_ydata(g)
    line_gate_unc.set_ydata(ug)
    fig_gate.canvas.draw_idle()

    return g, ug
    

gate_selector = interactive(
    update, 
    window_start = IntSlider(value=16, min=0, max=len(t)-window_width, step=1, description="window start: "), 
    window_width = IntSlider(value=25, min=1, max=len(t)//8, step=1, description="window width: "), 
    window_order = FloatSlider(value=4.6, min=0.0, max=10.0, step=0.2, description="window beta: ")
)

display(gate_selector)
gate_selector.children

In [ ]:
gate_array, gate_unc_array = gate_selector.result
gate_array

#### Non-Interactive Positioning

In [ ]:
window_kind = "kaiser"
window_order = 0.0  # 0 -> rect, +\infty -> gaussian
window_width = 11
window_start = 2

w = window(window_kind, window_order, window_width)
gate_array, gate_unc_array = gate(w, window_start, len(t))
gate_cov_array = np.diag(np.square(gate_unc_array))

In [ ]:
fig_gate, ax_gate = plt.subplots(nrows=2, figsize=(8, 8), tight_layout=True, sharex=True)

# setup secondary y-axis
ax2_gate = [None, None]
ax2_gate[0] = ax_gate[0].twinx()
ax2_gate[1] = ax_gate[1].twinx()

# plot signal and gate
ax_gate[0].plot(t, S11, label="signal", color="tab:gray")
ax2_gate[0].plot(t, gate_array, label="gate", color="r")

# plot uncertainty of signal and gate
ax_gate[1].plot(t, np.sqrt(np.diag(S11_cov)), label="signal unc", color="tab:gray")
ax2_gate[1].plot(t, gate_unc_array, label="gate unc", color="r")

# decorate
ax_gate[0].set_title("Time Domain (zoomed)")
ax_gate[0].legend(loc="upper right")
ax2_gate[0].legend(loc="upper center")
ax_gate[0].set_xlim((-0.1,1.5))
ax2_gate[1].set_ylim(ax_gate[1].get_ylim())

ax_gate[0].set_ylabel("signal [-]")
ax2_gate[0].set_ylabel("gate [-]")
ax_gate[1].set_ylabel("signal unc [-]")
ax2_gate[1].set_ylabel("gate unc [-]")
ax_gate[1].set_xlabel("t [ns]")

ax_gate[0].yaxis.get_label().set_color(line_signal.get_color())
ax2_gate[0].yaxis.get_label().set_color(line_gate.get_color())
ax_gate[1].yaxis.get_label().set_color(line_signal.get_color())
ax2_gate[1].yaxis.get_label().set_color(line_gate.get_color())

## Apply the Time-Gate

### Method 1 (multiplication and then DFT)
Applies the gate already in the time-domain and converts the result back to frequency domain. 

In [ ]:
# main calls
S11_gated = S11 * gate_array
S11_gated_cov = np.diag(gate_array) @ S11_cov @ np.diag(gate_array).T + np.diag(S11) @ gate_cov_array @ np.diag(S11).T
s11_gated_ri, s11_gated_ri_cov = GUM_DFT(S11_gated, S11_gated_cov)

#### Visualize Time Gated s11-Parameter

In [ ]:
# visualize result amp/phase
NN=len(s11_gated_ri)//2

# convert back to amplitude/phase representation
s11_gated_A, s11_gated_P, s11_gated_UAP = DFT2AmpPhase(s11_gated_ri, s11_gated_ri_cov)
s11_gated_UA = np.sqrt(np.diag(s11_gated_UAP)[:NN])
s11_gated_UP = np.sqrt(np.diag(s11_gated_UAP)[NN:])

fig_in, ax_in = plt.subplots(nrows=4, figsize=(8, 8), tight_layout=True)
ax_in[0].plot(f, s11_gated_A, label="s11 gated", color="tab:blue")
ax_in[1].plot(f, s11_gated_UA, label="s11 gated", color="tab:blue")
ax_in[2].plot(f, np.rad2deg(s11_gated_P), label="s11 gated", color="tab:blue")
ax_in[3].plot(f, np.rad2deg(s11_gated_UP), label="s11 gated", color="tab:blue")

ax_in[0].legend()
ax_in[0].grid(which="both", axis="both")
ax_in[0].set_title("Frequency Domain")
ax_in[0].set_ylabel("magnitude [-]")
ax_in[1].set_ylabel("magnitude unc [-]")
ax_in[2].set_ylabel("phase [°]")
ax_in[3].set_ylabel("phase unc [°]")
ax_in[3].set_xlabel("f [GHz]")


### Method 2 (DFT and then complex convolution)

Transforms gate into frequency domain and applies gate to the original (frequency domain) signal by using convolution operation. Output should match first method up to numerical precision. (Done with uncertainty evaluation by Monte Carlo, analytical uncertainty evaluation out of scope for now.)

In [ ]:
def make_twosided(x):
    # returns the twosided spectrum with f=0 at the start (default numpy style)
    # x = x_re + 1j * x_im 
    x_twosided = np.r_[x, np.conjugate(x[1:][::-1])]  # odd signal length
    #x_twosided = np.r_[x, np.conjugate(x[::-1])]  # even signal length (default assumption for rfft)
    return x_twosided

def make_onesided(x):
    # returns the twosided spectrum with f=0 at the start (default numpy style)
    # x = x_re + 1j * x_im, (size = 2*N - 1)
    N = (x.size + 1) // 2   # odd signal length
    #N = x.size // 2   # even signal length
    x_onesided = x[:N]
    return x_onesided

def complex_convolution_of_two_half_spectra(X, Y):
    # complex valued X, Y

    # transform into full spectra
    XX = make_twosided(X)
    YY = make_twosided(Y)

    # otherwise not strict ascending order (numpy default has f=0 at index 0, not in the middle)
    XX = np.fft.fftshift(XX) 
    YY = np.fft.fftshift(YY)

    # actual convolution
    RR = convolve1d(XX, YY, mode="wrap") / XX.size

    # undo shifting and make half spectrum
    R = make_onesided(np.fft.ifftshift(RR))

    return R

In [ ]:
# main calls
#gate_spectrum = np.fft.rfft(gate_array)
#s11_gated_conv = complex_convolution_of_two_half_spectra(ri2c(s11_ri), gate_spectrum)

# Monte Carlo of this main call

# draw gate and signal
def draw_samples(size, x1, x1_cov, x2, x2_cov):
    SAMPLES_X1 = np.random.multivariate_normal(x1, x1_cov, size)
    SAMPLES_X2 = np.random.multivariate_normal(x2, x2_cov, size)
    return (SAMPLES_X1, SAMPLES_X2)

# evaluate
n_runs = 1000
results = []
for s11_ri_mc, gate_array_mc in zip(*draw_samples(size=n_runs, x1=s11_ri, x1_cov=s11_ri_cov, x2=gate_array, x2_cov=gate_cov_array)):
    # main call
    gate_spectrum_tmp = np.fft.rfft(gate_array_mc)
    s11_gated_conv_tmp = complex_convolution_of_two_half_spectra(ri2c(s11_ri_mc), gate_spectrum_tmp)
    
    # save result
    results.append(c2ri(s11_gated_conv_tmp))

# extract mean and covariance
s11_gated_mcconv_ri = np.mean(results, axis=0)
s11_gated_mcconv_ri_cov = np.cov(results, rowvar=False)
s11_gated_mcconv = ri2c(s11_gated_mcconv_ri)

#### Visualize Time Gated s11-Parameter

In [ ]:
# visualize result amp/phase
NN=len(s11_gated_mcconv_ri)//2

# convert back to amplitude/phase representation
s11_gated_mcconv_A, s11_gated_mcconv_P, s11_gated_mcconv_UAP = DFT2AmpPhase(s11_gated_mcconv_ri, s11_gated_mcconv_ri_cov)
s11_gated_mcconv_UA = np.sqrt(np.diag(s11_gated_mcconv_UAP)[:NN])
s11_gated_mcconv_UP = np.sqrt(np.diag(s11_gated_mcconv_UAP)[NN:])

fig_in, ax_in = plt.subplots(nrows=4, figsize=(8, 8), tight_layout=True)
ax_in[0].plot(f, s11_gated_mcconv_A, label="s11 gated", color="tab:orange")
ax_in[1].plot(f, s11_gated_mcconv_UA, label="s11 gated", color="tab:orange")
ax_in[2].plot(f, np.rad2deg(s11_gated_mcconv_P), label="s11 gated", color="tab:orange")
ax_in[3].plot(f, np.rad2deg(s11_gated_mcconv_UP), label="s11 gated", color="tab:orange")

ax_in[0].legend()
ax_in[0].grid(which="both", axis="both")
ax_in[0].set_title("Frequency Domain")
ax_in[0].set_ylabel("magnitude [-]")
ax_in[1].set_ylabel("magnitude unc [-]")
ax_in[2].set_ylabel("phase [°]")
ax_in[3].set_ylabel("phase unc [°]")
ax_in[3].set_xlabel("f [GHz]")

## Comparison

### Original and Time-Gated Spectra of both methods

In [ ]:
fig_in, ax_in = plt.subplots(nrows=4, figsize=(8, 10), tight_layout=True)

ax_in[0].plot(f, s11_mag, label="s11 orig", color="tab:gray")
ax_in[0].plot(f, s11_gated_A, label="s11 gated", color="tab:blue", linewidth=5)
ax_in[0].plot(f, np.abs(s11_gated_mcconv), label="s11 gated (MC conv)", color="tab:orange")

ax_in[1].plot(f, s11_mag_unc, label="s11 orig unc", color="tab:gray")
ax_in[1].plot(f, s11_gated_UA, label="s11 gated", color="tab:blue", linewidth=5)
ax_in[1].plot(f, s11_gated_mcconv_UA, label="s11 gated (MC conv)", color="tab:orange")

ax_in[2].plot(f, np.rad2deg(s11_phase), label="s11 orig", color="tab:gray")
ax_in[2].plot(f, np.rad2deg(s11_gated_P), label="s11 gated", color="tab:blue", linewidth=5)
ax_in[2].plot(f, np.rad2deg(np.angle(s11_gated_mcconv)), label="s11 gated (MC conv)", color="tab:orange")

ax_in[3].plot(f, np.rad2deg(s11_phase_unc), label="s11 orig unc", color="tab:gray")
ax_in[3].plot(f, np.rad2deg(s11_gated_UP), label="s11 gated", color="tab:blue", linewidth=5)
ax_in[3].plot(f, np.rad2deg(s11_gated_mcconv_UP), label="s11 gated (MC conv)", color="tab:orange")


ax_in[0].legend()
ax_in[1].legend()
ax_in[2].legend()
ax_in[3].legend()
ax_in[0].set_title("Frequency Domain")
ax_in[0].set_ylabel("magnitude [-]")
ax_in[1].set_ylabel("magnitude unc [-]")
ax_in[2].set_ylabel("phase [°]")
ax_in[3].set_ylabel("phase unc [°]")
ax_in[3].set_xlabel("f [GHz]")
ax_in[1].set_yscale("log")
ax_in[3].set_yscale("log")
ax_in[3].set_ylim(1e-3, 1.5e1)

### Covariance Matrices

Covariances in Re/Im Representation of:

- input data
- output of method 1 (analytical uncertainty evaluation)
- output of method 2 (Monte Carlo uncertainty evaluation)

In [ ]:
# utility function for annotating plots
def annotate_real_imag_plot(ax, annotate_x=True, annotate_y=True):
    # define new tick positions
    labels_re = [0, 10, 20, 30]  # GHz
    labels_re = [0, 8, 16, 24]  # GHz
    labels_im = labels_re
    labels = labels_re + labels_im

    # define new labels for these positions
    ticks_re = [np.flatnonzero(f == l).item() for l in labels_re]
    ticks_im = [f.size + 1 + k for k in ticks_re]
    ticks = ticks_re + ticks_im

    # define colors for the labels to distinguish real and imag parts
    colors_re = ["k"] * len(ticks_re)
    colors_im = ["r"] * len(ticks_im)
    tick_colors = colors_re + colors_im

    if annotate_x:
        # xticks (label, position, color)
        ax.set_xticks(ticks=ticks, labels=labels)
        for ticklabel, c in zip(ax.get_xticklabels(), tick_colors):
            ticklabel.set_color(c)

        # axis label
        ax.set_xlabel("frequency [GHz]")

        # nice brace real part
        ax.annotate(
            "real",
            xy=(0.25, -0.01),
            xytext=(0.25, -0.10),
            fontsize=14,
            ha="center",
            va="bottom",
            xycoords="axes fraction",
            color="black",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

        # nice brace imag part
        ax.annotate(
            "imag",
            xy=(0.75, -0.01),
            xytext=(0.75, -0.10),
            fontsize=14,
            ha="center",
            va="bottom",
            xycoords="axes fraction",
            color="red",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

    if annotate_y:
        # yticks (label, position, color)
        ax.set_yticks(ticks=ticks, labels=labels)
        for ticklabel, c in zip(ax.get_yticklabels(), tick_colors):
            ticklabel.set_color(c)

        # axis label
        ax.set_ylabel("frequency [GHz]")

        # nice brace real part
        ax.annotate(
            "real",
            xy=(-0.01, 0.75),
            xytext=(-0.10, 0.75),
            fontsize=14,
            ha="left",
            va="center",
            xycoords="axes fraction",
            rotation=90,
            color="black",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

        # nice brace imag part
        ax.annotate(
            "imag",
            xy=(-0.01, 0.25),
            xytext=(-0.10, 0.25),
            fontsize=14,
            ha="left",
            va="center",
            xycoords="axes fraction",
            rotation=90,
            color="red",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

    return ax


In [ ]:
fig_cov, ax_cov = plt.subplots(nrows=3, figsize=(8, 25), tight_layout=True)
mini = max(1e-11, min(np.abs(s11_ri_cov).min(), np.abs(s11_gated_ri_cov).min()))
maxi = min(np.abs(s11_ri_cov).max(), np.abs(s11_gated_ri_cov).max())

cnorm = colors.SymLogNorm(vmin=-maxi, vmax=maxi, linthresh=mini)
img1 = ax_cov[0].imshow(s11_ri_cov, cmap="PuOr", norm=cnorm)
img2 = ax_cov[1].imshow(s11_gated_ri_cov, cmap="PuOr", norm=cnorm)
img3 = ax_cov[2].imshow(s11_gated_mcconv_ri_cov, cmap="PuOr", norm=cnorm)
fig_cov.colorbar(img1, ax=ax_cov[0])
fig_cov.colorbar(img2, ax=ax_cov[1])
fig_cov.colorbar(img3, ax=ax_cov[2])

ax_cov[0].set_title("Covariance of s11_ri")
ax_cov[0] = annotate_real_imag_plot(ax_cov[0])

ax_cov[1].set_title("Covariance of s11_gated_ri")
ax_cov[1] = annotate_real_imag_plot(ax_cov[1])

ax_cov[2].set_title("Covariance of s11_gated_mcconv_ri_cov")
ax_cov[2] = annotate_real_imag_plot(ax_cov[2])

#### Compare Gated Spectra and Covariance Matrices of analytical and Monte Carlo approach

In [ ]:
# visual comparison of the two methods:
fig_comp, ax_comp = plt.subplots(nrows=3, figsize=(8, 20), tight_layout=True)

# mean signed difference of values
ax_comp[0].plot(s11_gated_ri - s11_gated_mcconv_ri)
ax_comp[0].set_title("Mean Signed Difference of Gated Spectra")
ax_comp[0] = annotate_real_imag_plot(ax_comp[0], annotate_y=False)

# mean signed difference of covariance matrices
img4 = ax_comp[1].imshow(s11_gated_ri_cov - s11_gated_mcconv_ri_cov, cmap="PuOr", norm=cnorm)
fig_comp.colorbar(img4, ax=ax_comp[1])

ax_comp[1].set_title("Signed Difference of both Covariance Matrices")
ax_comp[1] = annotate_real_imag_plot(ax_comp[1])

# Kullback-Leibler divergence of covariance matrices
kl_div = special.kl_div(s11_gated_ri_cov, s11_gated_mcconv_ri_cov)
cnorm_kl = colors.LogNorm(vmin=1e-12, vmax=1e-8, clip=True)

img5 = ax_comp[2].imshow(kl_div, cmap="binary", norm=cnorm_kl)
fig_comp.colorbar(img5, ax=ax_comp[2])


ax_comp[2].set_title("Kullback-Leibler divergence of both Covariance Matrices")
ax_comp[2] = annotate_real_imag_plot(ax_comp[2])